In [22]:

from movement import sample_data
from ethograph import minimal_dt_from_pose
import sleap_io as sio
import xarray as xr
from matplotlib import pyplot as plt

import movement.kinematics as kin
from movement import sample_data
from movement.filtering import rolling_filter
from movement.plots import plot_centroid_trajectory
from movement.kinematics import compute_velocity

ds = sample_data.fetch_dataset(
    "DLC_rotating-mouse_eye-tracking_stim-black.predictions.h5",
    with_video=True,
)

positions = ds["position"]

eye_midpoint = positions.sel(keypoints=["eye-L", "eye-R"]).mean("keypoints")
positions_norm = positions - eye_midpoint
pupil_centroid = (
    positions_norm.sel(keypoints=["pupil-L", "pupil-R"])
    .mean("keypoints")
    .assign_coords({"keypoints": "pupil-C"})
)
positions_norm = xr.concat([positions_norm, pupil_centroid], dim="keypoints")




pupil_velocity = compute_velocity(positions_norm.sel(keypoints="pupil-C"))
pupil_diameter: xr.DataArray = kin.compute_pairwise_distances(
    positions_norm, dim="keypoints", pairs={"pupil-L": "pupil-R"}
)
pupil_diameter.name = "pupil diameter"
window_len = 80
mean_filter = rolling_filter(
    pupil_diameter, window=window_len, statistic="mean"
)
mdn_filter = rolling_filter(pupil_diameter, window_len, statistic="median")

ds["pupil_velocity"] = pupil_velocity
ds["pupil_diameter"] = pupil_diameter
ds["pupil_diameter_filt"] = mdn_filter

for var in ["pupil_velocity", "pupil_diameter", "pupil_diameter_filt"]:
    ds[var].attrs["type"] = "feature"
    ds[var].attrs["individuals"] = "individual_0"
    ds[var].attrs["keypoints"] = "pupil-C"

In [23]:
ds.pupil_velocity

<xarray.DataArray 'pupil_velocity' (time: 7000, space: 2, individuals: 1)> Size: 112kB
8.481 43.94 -9.734 7.258 -6.266 -13.21 ... 3.302 -0.3722 -17.18 16.98 -9.204
Coordinates:
  * time         (time) float64 56kB 0.0 0.025 0.05 0.075 ... 174.9 174.9 175.0
  * space        (space) <U1 8B 'x' 'y'
  * individuals  (individuals) <U12 48B 'individual_0'
Attributes:
    type:         feature
    individuals:  individual_0
    keypoints:    pupil-C

In [ ]:
dt = minimal_dt_from_pose(video_path=r"C:\Users\aksel\Documents\Code\EthoGraph\data\pupil\video.mp4",
                          fps=60,
                          pose_path=r"C:\Users\aksel\Documents\Code\EthoGraph\data\pupil\pose.h5",
                          source_software="DeepLabCut")
                          
                          

dt.to_netcdf(r"C:\Users\aksel\Documents\Code\EthoGraph\data\pupil\pupil.nc")